<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-файл-с-данными-и-изучите-общую-информацию" data-toc-modified-id="Откройте-файл-с-данными-и-изучите-общую-информацию-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте файл с данными и изучите общую информацию</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответьте-на-вопросы" data-toc-modified-id="Ответьте-на-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответьте на вопросы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
data=pd.read_csv('data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

На основании метода info() в двух столбцах вижу пропущенные значения. На основе ознакомления с таблицей вижу сразу следующие недочеты в данных, которые могут помешать дальнейшей обработке данных: в трудовом стаже указаны отрицательные значения, некоторые значения чрезмерно велики, в строковых столбцах используются разные регистры, в целях предоставления кредита нет категоризации. 

##  Предобработка данных

### Обработка пропусков

In [4]:
data['days_employed_module']=abs(data['days_employed']) # вычисление модуля числа с помощью метода abs()

print ('Количество пропусков в столбце "стаж" до обработки пропусков: ', data['days_employed_module'].isna().sum())
print ('Количество пропусков в столбце "ежемесячный доход" до обработки пропусков: ', data['total_income'].isna().sum())
print('----')

avg_days_employed=data['days_employed_module'].median() #медианное значение занятости
#замена пропущенных значений ранее вычесленным медианным значением:
data['days_employed_module']=data['days_employed_module'].fillna(value=avg_days_employed)

# замена пропущенных значений медианными для зароботка:
for position in data ['income_type'].unique():
    median=data.loc[data['income_type'] == position,'total_income'].median()
    print (position, median)
    data.loc[(data['total_income'].isna())&(data['income_type']==position), 'total_income']=median

print ('Количество пропусков в столбце "стаж" после обработки пропусков: ', data['days_employed_module'].isna().sum())
print ('Количество пропусков в столбце "ежемесячный доход" после обработки пропусков: ', data['total_income'].isna().sum())


Количество пропусков в столбце "стаж" до обработки пропусков:  2174
Количество пропусков в столбце "ежемесячный доход" до обработки пропусков:  2174
----
сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995
Количество пропусков в столбце "стаж" после обработки пропусков:  0
Количество пропусков в столбце "ежемесячный доход" после обработки пропусков:  0


**Вывод**

Пропущенные значения присутствуют у разных возрастных групп, кредит был взят на разные цели, на основании этого предполагаю, что данные просто не заполнили (пропуски в двух столбцах - стаж и месячный доход). Перед тем, как заполнить найденные пропуски, перевела отрицательные числа в положительные, для адекватности данных. Также я заметила, что некоторые значения трудовой занятости чрезмерно велики, т.к. я не знаю, чем вызвана эта ошибка (вероятно случайная), решила заполнить пропущенные значения медианными значениями, т.к. среднее значения из-за артефактов не будет отражать реальные цифры (такой результат может получится только после очередной пенсионной рефоры). После убедила, что все пропуски заполнены.

### Замена типа данных

In [5]:
data['days_employed_module']=data['days_employed_module'].astype('int')
data['total_income']=data['total_income'].astype('int')
data.info() #убедилась, что тип данных изменен

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21525 non-null  int64  
 1   days_employed         19351 non-null  float64
 2   dob_years             21525 non-null  int64  
 3   education             21525 non-null  object 
 4   education_id          21525 non-null  int64  
 5   family_status         21525 non-null  object 
 6   family_status_id      21525 non-null  int64  
 7   gender                21525 non-null  object 
 8   income_type           21525 non-null  object 
 9   debt                  21525 non-null  int64  
 10  total_income          21525 non-null  int32  
 11  purpose               21525 non-null  object 
 12  days_employed_module  21525 non-null  int32  
dtypes: float64(1), int32(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Данные приведены к нужному типу.

### Обработка дубликатов

In [6]:
print(data['education'].value_counts(), data['income_type'].value_counts(), data['family_status'].value_counts())
data['education_lowercase']=data['education'].str.lower() #привела все записи к одноу регистру
print(data['education_lowercase'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64 сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64 женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                28

**Вывод**

Для начала проверила уникальные значения в трех столбцах - образование, тип занятости, семейный статус. Цель кредита не стала проверять, потому что там необходимо работать с леммами. В столбце образование дубликаты из-за использования разного регистра при вводе данных, либо из-за того, что склейка базы произведена из разных источников, где требования к регистру была разная. Для устранения таких дубликатов использую метод str.lower() для приведения всех значений к единому, данные после обработки сохранила в новый столбец и проверила на дубликаты - их не осталось!

### Лемматизация

In [7]:
m = Mystem()
text=' '.join(data['purpose'])
lemmas = m.lemmatize(text)

print(Counter(lemmas)) 

Counter({' ': 55201, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'подержать': 858, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'приобретение': 462, 'профильный': 436, 'подержанный': 110, '\n': 1})


**Вывод**

Лемматизацию выполнила с помощью библиотеки pymystem3, т.к. мне этот метод показался наиболее наглядным и с ним удобнее работать. По итогам выполнения кода видно, что наиболее часто встречающиеся цели кредита это недвижимость,  покупка авто, образование, свадьба и ремонт

### Категоризация данных

In [8]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'автомобиль'
        elif 'жил' in row or 'недвиж' in row:
            return 'недвижимость'
        elif 'свад' in row:
            return 'свадьба'
        elif 'образов' in row:
            return 'образование'
    except:
        return 'нет категории'

In [9]:
data['purpose_ctg']=data['purpose'].apply(categorize_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_module,education_lowercase,purpose_ctg
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,высшее,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,среднее,автомобиль
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,среднее,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,среднее,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,340266,среднее,свадьба


**Вывод**

На основании использования предыдущего метода определила основные категории и присвоила их подходящим значениям.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [14]:
data_grouped_child=data.pivot_table(index='children', values='debt', aggfunc='sum')
data_grouped_child['count_debt']=data.pivot_table(index='children', values='debt', aggfunc='count')
data_grouped_child['percent_debt']=(data_grouped_child['debt']/data_grouped_child['count_debt'])*100
data_grouped_child.sort_values('percent_debt',ascending=[False])

,debt,count_debt,percent_debt
children,,,
20,8,76,10.526316
4,4,41,9.756098
2,194,2055,9.440389
1,444,4818,9.215442
3,27,330,8.181818
0,1063,14149,7.512898
-1,1,47,2.127660
5,0,9,0.000000


**Вывод**

Самая большая просрочка по платежам у многодетных семей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
data_grouped_marital=data.pivot_table(index='family_status', values='debt', aggfunc='sum')
data_grouped_marital['count_debt']=data.pivot_table(index='family_status', values='debt', aggfunc='count')
data_grouped_marital['percent_debt']=(data_grouped_marital['debt']/data_grouped_marital['count_debt'])*100
data_grouped_marital.sort_values('percent_debt',ascending=[False])

,debt,count_debt,percent_debt
family_status,,,
Не женат / не замужем,274,2813,9.740491
гражданский брак,388,4177,9.288963
женат / замужем,931,12380,7.520194
в разводе,85,1195,7.112971
вдовец / вдова,63,960,6.562500


**Вывод**

Посмотрев на сводную таблицу, можно увидеть, что доля просроченых кредитов больше у людей, которые находятся в гражданском браке или не женаты. Чаще всего возращают кредиты вдовцы и вдовы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
data['income_cat']=pd.qcut(data['total_income'], q=4)
data_grouped_income=data.pivot_table(index='income_cat', values='debt', aggfunc='sum')
data_grouped_income['count_debt']=data.pivot_table(index='income_cat', values='debt', aggfunc='count')
data_grouped_income['percent_debt']=(data_grouped_income['debt']/data_grouped_income['count_debt'])*100
data_grouped_income.sort_values('percent_debt',ascending=[False])

,debt,count_debt,percent_debt
income_cat,,,
"(107798.0, 142594.0]",483,5523,8.745247
"(142594.0, 195549.0]",445,5239,8.493987
"(20666.999, 107798.0]",427,5382,7.933854
"(195549.0, 2265604.0]",386,5381,7.173388


**Вывод**

Уровень дохода не значительно влияет на кредитные выплаты в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [17]:
data_grouped_purpose=data.pivot_table(index='purpose_ctg', values='debt', aggfunc='sum')
data_grouped_purpose['count_debt']=data.pivot_table(index='purpose_ctg', values='debt', aggfunc='count')
data_grouped_purpose['percent_debt']=(data_grouped_purpose['debt']/data_grouped_purpose['count_debt'])*100
data_grouped_purpose.sort_values('percent_debt',ascending=[False])

,debt,count_debt,percent_debt
purpose_ctg,,,
автомобиль,403,4315,9.339513
образование,370,4022,9.199403
свадьба,186,2348,7.921635
недвижимость,782,10840,7.214022


**Вывод**

Чаще других просрачивают платежи за образование и на приобретение автомобиля.

## Общий вывод

Больше всего на невозврат кредитных средств в срок влияет количество детей (чем больше детей, тем риск выше), уровень дохода практически не влияет при этом. Незначительное влияение оказывает цель кредита (в зоне риска автомобильные кредиты и образование) и семейное положение (люди не состоящие в отношениеях или состоящие в незарегистрированных отношениях более необязательные).